In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import normalize
import fanc
from scipy.spatial import distance
from meshparty import trimesh_io, meshwork, mesh_filters
import navis
import pcg_skel
from fanc import auth

client = fanc.get_caveclient()

In [65]:
try:
    version_number = client.materialize.version
except Exception as e:
    print(f"Failed to retrieve version number: {e}")
    version_number = 1  # デフォルトのバージョン番号を設定

def post_neurons_2(neuron_a, sc_limit):
    """
    Find downstream neurons of the neuron_a, which has more than 'sc_limit' synapses with neuron_a.
        Args: 
            neuron_a (int) : segment ID # of a neuron of interest.
            sc_limit (int) : minimum synaps count for cutting off.
        Return: a table of downstream neurons and synaps counts as DataFrame."""
    # Get synapse position table of downstream neurons of neuron_a. Each row represents one synaps position.
    post_df = client.materialize.synapse_query(pre_ids = neuron_a, materialization_version=version_number)
    if len(post_df) == 0:
        print('Check if ID# is current or No downstream neurons of ' + str(neuron_a))
    # Choose oly required columns.
    post_df2 = post_df[['id','pre_pt_root_id', 'post_pt_root_id']]
    # Data type of id#s will be changed from 'int' to 'str' to avoid any problem caused by transformation etc.
    post_df2[['id','pre_pt_root_id','post_pt_root_id' ]] = post_df2[['id','pre_pt_root_id','post_pt_root_id' ]].astype('str')
    # Transform # of rows to synapse count. Then add these numbers to a new column 'synaps_count' and sort them as descending.
    synaps_count = post_df2.groupby('post_pt_root_id').transform(len)['id']
    post_df2['synaps_count'] = synaps_count
    post_df3 = post_df2.sort_values('synaps_count', ascending=False)
    # Make one row represent one downstream neuron. Clean the table by dropping unnecessary columns. 
    # Cut off neurons which have syanpse count fewer than 'sc_limit'.
    post_df4 = post_df3.drop_duplicates(subset='post_pt_root_id', keep='first')
    post_df5 = post_df4.loc[post_df4['synaps_count'] >= sc_limit].drop (columns = 'id')
    
    return post_df5


In [66]:
def pre_neurons(neuron_a, sc_limit):
    """
    Find upstream neurons of the neuron_a, which has more than 'sc_limit' synapses with neuron_a.
        Args: 
            neuron_a (int) : segment ID # of a neuron of interest.
            sc_limit (int) : minimum synaps count for cutting off.

        Return: a table of upstream neurons and synaps counts as DataFrame."""            
    # Get synapse position table of upstream neurons of neuron_a. Each row represents one synaps position.    
    pre_df = client.materialize.synapse_query(post_ids = neuron_a, materialization_version=version_number)
    if len(pre_df) == 0:
        print('Check if ID# is current or No upstream neurons of ' + str(neuron_a))
    # Choose only required columns.
    pre_df2 = pre_df[['id','pre_pt_root_id', 'post_pt_root_id']]
    # Data type of id#s will be changed from 'int' to 'str' to avoid any problem caused by transformation etc.
    pre_df2[['id','pre_pt_root_id','post_pt_root_id' ]] = pre_df2[['id','pre_pt_root_id','post_pt_root_id' ]].astype('str')
    # Transform # of rows to synapse count. Then add these numbers to a new column 'synaps_count' and sort them as descending.
    synaps_count = pre_df2.groupby('pre_pt_root_id').transform(len)['id']
    pre_df2['synaps_count'] = synaps_count
    pre_df3 = pre_df2.sort_values('synaps_count', ascending=False)
    # Make one row represent one downstream neuron. Clean the table by dropping unnecessary columns. 
    # Cut off neurons which have syanpse count fewer than 'sc_limit'.
    pre_df4 = pre_df3.drop_duplicates(subset='pre_pt_root_id', keep='first')
    pre_df5 = pre_df4.loc[pre_df4['synaps_count'] >= sc_limit].drop (columns = 'id')
    
    return pre_df5

In [67]:
# Function to create skeleton
def get_pcg_skeleton(segid, **kwargs):
    try:
        if 'client' not in kwargs:
            kwargs['client'] = auth.get_caveclient()
        return pcg_skel.pcg_skeleton(segid, **kwargs)
    except Exception as e:
        raise Exception(f"Error retrieving skeleton for segment ID {segid}: {e}")

In [68]:
# Function to flip skeleton horizontally
def flip_skeleton_horizontally(skeleton):
    flipped_vertices = skeleton.vertices.copy()
    flipped_vertices[:, 0] = -flipped_vertices[:, 0]
    skeleton.vertices = flipped_vertices
    return skeleton

In [69]:
# Function to save skeleton in SWC format
def save_swc(skeleton, filename):
    with open(filename, 'w') as file:
        for i, vertex in enumerate(skeleton.vertices):
            line = f"{i+1} 3 {vertex[0]} {vertex[1]} {vertex[2]} 1.0 -1\n"
            file.write(line)

In [70]:
print(post_neurons_2)  # グループ化する前に、データフレームの中身を確認

<function post_neurons_2 at 0x00000229FAF10360>


In [71]:
# set up threashold of synapse count. Only more than this s.c connection is considered to be meaningful connection.
sc_limit = 20

neuron_1 = 648518346475464576
#IN_011_left change the segment ID

# Find downstream neurons of neuron_1 using function 'post_neurons_2'.
neuron_1_post_df = post_neurons_2(neuron_1,sc_limit )
neuron_1_post_df

# Create output directory if it doesn't exist
output_dir = "left_swc_files"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

C:\Users\jtcn0\AppData\Local\Temp\ipykernel_28072\1624383906.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_df2[['id','pre_pt_root_id','post_pt_root_id' ]] = post_df2[['id','pre_pt_root_id','post_pt_root_id' ]].astype('str')
C:\Users\jtcn0\AppData\Local\Temp\ipykernel_28072\1624383906.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_df2['synaps_count'] = synaps_count


In [72]:
# Assuming neuron_1_post_df is already defined and contains the necessary data
# Get the list of post_pt_root_ids
post_neuron_ids = neuron_1_post_df['post_pt_root_id'].tolist()

# Download and save SWC files for each post_neuron
for idx, neuron_id in enumerate(post_neuron_ids):
    try:
        skeleton = get_pcg_skeleton(int(neuron_id))
        filename = os.path.join(output_dir, f"left_neuron_{neuron_id}.swc")
        save_swc(skeleton, filename)
        print(f"Saved SWC file for neuron {neuron_id} at {filename}")
    except Exception as e:
        print(f"Failed to retrieve or save skeleton for neuron {neuron_id}: {e}")

Saved SWC file for neuron 648518346490321847 at left_swc_files\left_neuron_648518346490321847.swc
Saved SWC file for neuron 648518346504996486 at left_swc_files\left_neuron_648518346504996486.swc
Saved SWC file for neuron 648518346490281143 at left_swc_files\left_neuron_648518346490281143.swc
Saved SWC file for neuron 648518346510036026 at left_swc_files\left_neuron_648518346510036026.swc
Saved SWC file for neuron 648518346485677843 at left_swc_files\left_neuron_648518346485677843.swc
Saved SWC file for neuron 648518346488666607 at left_swc_files\left_neuron_648518346488666607.swc
Saved SWC file for neuron 648518346489544793 at left_swc_files\left_neuron_648518346489544793.swc
Saved SWC file for neuron 648518346500750697 at left_swc_files\left_neuron_648518346500750697.swc
Saved SWC file for neuron 648518346500682165 at left_swc_files\left_neuron_648518346500682165.swc
Saved SWC file for neuron 648518346515968074 at left_swc_files\left_neuron_648518346515968074.swc
Saved SWC file for n

In [73]:
neuron_1 = 648518346488820970
# Find downstream neurons of neuron_1 using function 'post_neurons_2'
neuron_1_post_df = post_neurons_2(neuron_1, sc_limit)
print(neuron_1_post_df)

# Create output directory if it doesn't exist
output_dir = "right_swc_files"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Assuming neuron_1_post_df is already defined and contains the necessary data
# Get the list of post_pt_root_ids
post_neuron_ids = neuron_1_post_df['post_pt_root_id'].tolist()

C:\Users\jtcn0\AppData\Local\Temp\ipykernel_28072\1624383906.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_df2[['id','pre_pt_root_id','post_pt_root_id' ]] = post_df2[['id','pre_pt_root_id','post_pt_root_id' ]].astype('str')


           pre_pt_root_id     post_pt_root_id  synaps_count
6937   648518346488820970  648518346486765539           102
149    648518346488820970  648518346502609063            77
11915  648518346488820970  648518346480562390            54
5609   648518346488820970  648518346513926169            54
11450  648518346488820970  648518346480301526            49
6391   648518346488820970  648518346501370200            42
12694  648518346488820970  648518346482834729            35
8586   648518346488820970  648518346486930145            34
9664   648518346488820970  648518346490276279            31
8721   648518346488820970  648518346489556823            30
10246  648518346488820970  648518346479591387            29
6411   648518346488820970  648518346495587766            28
10670  648518346488820970  648518346488038612            27
12708  648518346488820970  648518346494168178            25
9683   648518346488820970  648518346498409009            24
10457  648518346488820970  6485183464971

C:\Users\jtcn0\AppData\Local\Temp\ipykernel_28072\1624383906.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_df2['synaps_count'] = synaps_count


In [74]:
# Download, flip, and save SWC files for each post_neuron
for idx, neuron_id in enumerate(post_neuron_ids):
    try:
        skeleton = get_pcg_skeleton(int(neuron_id))
        flipped_skeleton = flip_skeleton_horizontally(skeleton)
        filename = os.path.join(output_dir, f"right_flipped_neuron_{neuron_id}.swc")
        save_swc(flipped_skeleton, filename)
        print(f"Saved SWC file for flipped neuron {neuron_id} at {filename}")
    except Exception as e:
        print(f"Failed to retrieve or save skeleton for neuron {neuron_id}: {e}")

Saved SWC file for flipped neuron 648518346486765539 at right_swc_files\right_flipped_neuron_648518346486765539.swc
Saved SWC file for flipped neuron 648518346502609063 at right_swc_files\right_flipped_neuron_648518346502609063.swc
Saved SWC file for flipped neuron 648518346480562390 at right_swc_files\right_flipped_neuron_648518346480562390.swc
Saved SWC file for flipped neuron 648518346513926169 at right_swc_files\right_flipped_neuron_648518346513926169.swc
Saved SWC file for flipped neuron 648518346480301526 at right_swc_files\right_flipped_neuron_648518346480301526.swc
Saved SWC file for flipped neuron 648518346501370200 at right_swc_files\right_flipped_neuron_648518346501370200.swc
Saved SWC file for flipped neuron 648518346482834729 at right_swc_files\right_flipped_neuron_648518346482834729.swc
Saved SWC file for flipped neuron 648518346486930145 at right_swc_files\right_flipped_neuron_648518346486930145.swc
Saved SWC file for flipped neuron 648518346490276279 at right_swc_files\

In [75]:
import os
import pandas as pd

# Define input and output directories for SWC files
input_dirs = [
    r"C:\Users\jtcn0\MyProject\left_swc_files",
    r"C:\Users\jtcn0\MyProject\right_swc_files"
]

# Process all SWC files in the input directories
for input_dir in input_dirs:
    for filename in os.listdir(input_dir):
        if filename.endswith(".swc"):
            file_path = os.path.join(input_dir, filename)
            
            try:
                # Read the SWC file
                df = pd.read_csv(file_path, sep=' ', header=None, comment='%', engine='python')
                
                # Replace all -1 parent nodes with 1
                df[6] = df[6].replace(-1, 1)
                
                # Save the modified SWC file
                output_path = os.path.join(input_dir, filename)
                df.to_csv(output_path, sep=' ', header=False, index=False)
                
                print(f"Modified SWC file saved to: {output_path}")
            except Exception as e:
                print(f"Failed to process file {filename}: {e}")

Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346474776002.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346478277845.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346479186384.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346479598034.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346484674205.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346485677843.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346486393490.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346486992636.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346487005665.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_

In [76]:
#It's possible to make codes that only changes the first 1 into -1.

In [77]:
import os
import pandas as pd

# Define input directories for SWC files
input_dirs = [
    r"C:\Users\jtcn0\MyProject\left_swc_files",
    r"C:\Users\jtcn0\MyProject\right_swc_files"
]

# Process all SWC files in the input directories
for input_dir in input_dirs:
    for filename in os.listdir(input_dir):
        if filename.endswith(".swc"):
            file_path = os.path.join(input_dir, filename)
            
            try:
                # Read the SWC file
                df = pd.read_csv(file_path, sep=' ', header=None, comment='%', engine='python')
                
                # Check and replace the parent node of the first row
                if df.iloc[0, 6] == 1:  # If the 7th column (index 6) is 1
                    df.iloc[0, 6] = -1   # Change it to -1
                
                # Save the modified SWC file
                output_path = os.path.join(input_dir, filename)
                df.to_csv(output_path, sep=' ', header=False, index=False)
                
                print(f"Modified SWC file saved to: {output_path}")
            except Exception as e:
                print(f"Failed to process file {filename}: {e}")


Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346474776002.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346478277845.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346479186384.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346479598034.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346484674205.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346485677843.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346486393490.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346486992636.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_neuron_648518346487005665.swc
Modified SWC file saved to: C:\Users\jtcn0\MyProject\left_swc_files\left_